# AI-Driven Early Prediction of Pulmonary Fibrosis Using Deep Learning


In [2]:
import os
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
import warnings

warnings.filterwarnings('ignore')

# ==========================================
# CONFIGURATION
# ==========================================
CONFIG = {
    # Neural Net Config
    "nn_lr": 2e-3,
    "nn_weight_decay": 3e-5,
    "nn_batch_size": 64,
    "nn_epochs": 300,
    "nn_patience": 50,
    
    # LightGBM Config
    "lgb_params": {
        'objective': 'quantile',
        'metric': 'quantile',
        'alpha': 0.5,  # Will train 3 models for q20, q50, q80
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1,
        'max_depth': 6,
        'min_child_samples': 20,
        'reg_alpha': 0.1,
        'reg_lambda': 0.1
    },
    "lgb_rounds": 500,
    "lgb_early_stopping": 50,
    
    # General
    "n_folds": 5,
    "quantiles": [0.2, 0.5, 0.8],
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "data_dir": "../input/osic-pulmonary-fibrosis-progression",
    "biomarker_path": "../input/feature-extraction-u-net-segmentation/master_dataset.csv",
    "seed": 42,
    
    # Ensemble weights (tuned during validation)
    "ensemble_nn_weight": 0.5,  # Will be optimized
    "ensemble_lgb_weight": 0.5
}

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(CONFIG['seed'])

# ==========================================
# DATA PREPROCESSING (Same as before)
# ==========================================
def preprocess_data_no_leakage(config):
    """Zero look-ahead bias preprocessing"""
    clinical_df = pd.read_csv(f"{config['data_dir']}/train.csv")
    biomarkers_df = pd.read_csv(config['biomarker_path'])
    
    image_features = [
        'lung_vol_ml', 'hu_mean', 'hu_std', 'hu_skew', 'hu_kurt',
        'glcm_contrast', 'glcm_homogeneity', 'glcm_energy', 'glcm_correlation'
    ]
    
    cols_to_keep = ['Patient'] + [c for c in image_features if c in biomarkers_df.columns]
    biomarkers_clean = biomarkers_df[cols_to_keep].drop_duplicates(subset=['Patient'])
    
    train = clinical_df.merge(biomarkers_clean, on='Patient', how='inner')
    train['Weeks'] = train['Weeks'].astype(int)
    train.sort_values(['Patient', 'Weeks'], inplace=True)
    
    baseline = train.groupby('Patient').first().reset_index()
    baseline = baseline[['Patient', 'FVC', 'Percent']].rename(
        columns={'FVC': 'Base_FVC', 'Percent': 'Base_Percent'}
    )
    train = train.merge(baseline, on='Patient', how='left')
    
    base_weeks = train.groupby('Patient')['Weeks'].min().reset_index().rename(
        columns={'Weeks': 'Base_Week'}
    )
    train = train.merge(base_weeks, on='Patient', how='left')
    train['Relative_Weeks'] = train['Weeks'] - train['Base_Week']
    
    train['FVC_Ratio'] = train['FVC'] / train['Base_FVC']
    
    train['Weeks_squared'] = train['Relative_Weeks'] ** 2
    train['Weeks_cubed'] = train['Relative_Weeks'] ** 3
    train['Week_Decay'] = 1 - np.exp(-train['Relative_Weeks'] / 52.0)
    
    train['FVC_Week_Interaction'] = train['Base_FVC'] * train['Relative_Weeks']
    train['Age_Week_Interaction'] = train['Age'] * train['Relative_Weeks']
    train['Percent_Week_Interaction'] = train['Base_Percent'] * train['Relative_Weeks']
    
    available_img_feats = [c for c in image_features if c in train.columns]
    
    if 'lung_vol_ml' in train.columns:
        train['LungVol_FVC_Ratio'] = train['lung_vol_ml'] / (train['Base_FVC'] + 1e-6)
        train['LungVol_Week_Interaction'] = train['lung_vol_ml'] * train['Relative_Weeks']
    
    if 'hu_mean' in train.columns:
        train['HU_Week_Interaction'] = train['hu_mean'] * train['Relative_Weeks']
    
    train['Base_FVC_Raw'] = train['Base_FVC']
    
    train['Sex'] = train['Sex'].apply(lambda x: 1 if x == 'Male' else 0)
    train['Smk_Ex'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Ex-smoker' else 0)
    train['Smk_Cur'] = train['SmokingStatus'].apply(lambda x: 1 if x == 'Currently smokes' else 0)
    
    num_cols = ['Age', 'Base_Percent', 'Relative_Weeks', 'Weeks_squared', 
                'Weeks_cubed', 'Week_Decay'] + available_img_feats
    
    interaction_cols = ['FVC_Week_Interaction', 'Age_Week_Interaction', 
                       'Percent_Week_Interaction']
    
    if 'LungVol_FVC_Ratio' in train.columns:
        interaction_cols.extend(['LungVol_FVC_Ratio', 'LungVol_Week_Interaction'])
    if 'HU_Week_Interaction' in train.columns:
        interaction_cols.append('HU_Week_Interaction')
    
    feature_cols = num_cols + interaction_cols + ['Sex', 'Smk_Ex', 'Smk_Cur']
    
    print(f"✅ Preprocessing: {train.shape[0]} samples, {len(feature_cols)} features")
    
    return train, feature_cols, num_cols + interaction_cols

# ==========================================
# NEURAL NETWORK MODEL
# ==========================================
class EnhancedQuantileMLP(nn.Module):
    def __init__(self, input_dim, quantiles, dropout=0.25):
        super().__init__()
        h1, h2, h3 = 256, 128, 64
        
        self.net = nn.Sequential(
            nn.Linear(input_dim, h1), nn.BatchNorm1d(h1), nn.LeakyReLU(0.1), nn.Dropout(dropout),
            nn.Linear(h1, h2), nn.BatchNorm1d(h2), nn.LeakyReLU(0.1), nn.Dropout(dropout),
            nn.Linear(h2, h3), nn.BatchNorm1d(h3), nn.LeakyReLU(0.1), nn.Dropout(dropout),
            nn.Linear(h3, len(quantiles))
        )
    
    def forward(self, x):
        return self.net(x)

def quantile_loss(preds, target, quantiles):
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i].unsqueeze(1)
        loss = torch.max((q-1) * errors, q * errors)
        losses.append(loss)
    return torch.mean(torch.sum(torch.cat(losses, dim=1), dim=1))

# ==========================================
# METRICS
# ==========================================
def calculate_metrics(y_true_fvc, q_preds_ratio, baseline_fvc):
    q20 = q_preds_ratio[:, 0] * baseline_fvc
    q50 = q_preds_ratio[:, 1] * baseline_fvc
    q80 = q_preds_ratio[:, 2] * baseline_fvc
    
    sigma = q80 - q20
    sigma_clipped = np.maximum(sigma, 70)
    
    delta = np.minimum(np.abs(y_true_fvc - q50), 1000)
    lll = - (np.sqrt(2) * delta / sigma_clipped) - np.log(np.sqrt(2) * sigma_clipped)
    
    mse = mean_squared_error(y_true_fvc, q50)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true_fvc, q50)
    r2 = r2_score(y_true_fvc, q50)
    
    return {'lll': np.mean(lll), 'mse': mse, 'rmse': rmse, 'mae': mae, 'r2': r2}

# ==========================================
# NEURAL NETWORK TRAINING
# ==========================================
def train_nn_fold(train_data, val_data, features, scale_cols, fold):
    """Train single fold of neural network"""
    feature_scaler = StandardScaler()
    ratio_scaler = StandardScaler()
    
    train_data_scaled = train_data.copy()
    val_data_scaled = val_data.copy()
    
    train_data_scaled[scale_cols] = feature_scaler.fit_transform(train_data[scale_cols])
    train_data_scaled['FVC_Ratio_Scaled'] = ratio_scaler.fit_transform(train_data[['FVC_Ratio']])
    
    val_data_scaled[scale_cols] = feature_scaler.transform(val_data[scale_cols])
    val_data_scaled['FVC_Ratio_Scaled'] = ratio_scaler.transform(val_data[['FVC_Ratio']])
    
    X_train = torch.tensor(train_data_scaled[features].values, dtype=torch.float32).to(CONFIG['device'])
    y_train = torch.tensor(train_data_scaled['FVC_Ratio_Scaled'].values, dtype=torch.float32).unsqueeze(1).to(CONFIG['device'])
    
    X_val = torch.tensor(val_data_scaled[features].values, dtype=torch.float32).to(CONFIG['device'])
    
    model = EnhancedQuantileMLP(len(features), CONFIG['quantiles']).to(CONFIG['device'])
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG['nn_lr'], weight_decay=CONFIG['nn_weight_decay'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=20, verbose=False)
    
    best_lll = -float('inf')
    best_preds = None
    patience = 0
    
    for epoch in range(CONFIG['nn_epochs']):
        model.train()
        optimizer.zero_grad()
        preds = model(X_train)
        loss = quantile_loss(preds, y_train, CONFIG['quantiles'])
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_preds_scaled = model(X_val)
            val_preds_ratio = ratio_scaler.inverse_transform(val_preds_scaled.cpu().numpy())
            metrics = calculate_metrics(val_data['FVC'].values, val_preds_ratio, val_data['Base_FVC_Raw'].values)
            
        scheduler.step(metrics['lll'])
        
        if metrics['lll'] > best_lll:
            best_lll = metrics['lll']
            best_preds = val_preds_ratio.copy()
            patience = 0
            torch.save({'model': model.state_dict(), 'feature_scaler': feature_scaler, 
                       'ratio_scaler': ratio_scaler}, f"nn_fold{fold}.pth")
        else:
            patience += 1
            
        if patience >= CONFIG['nn_patience']:
            break
    
    return best_preds, best_lll

# ==========================================
# LIGHTGBM TRAINING
# ==========================================
def train_lgb_fold(train_data, val_data, features, fold):
    """Train LightGBM for all 3 quantiles"""
    lgb_preds = []
    
    for q_idx, quantile in enumerate(CONFIG['quantiles']):
        params = CONFIG['lgb_params'].copy()
        params['alpha'] = quantile
        
        dtrain = lgb.Dataset(train_data[features], label=train_data['FVC_Ratio'])
        dval = lgb.Dataset(val_data[features], label=val_data['FVC_Ratio'], reference=dtrain)
        
        model = lgb.train(
            params,
            dtrain,
            num_boost_round=CONFIG['lgb_rounds'],
            valid_sets=[dval],
            callbacks=[lgb.early_stopping(CONFIG['lgb_early_stopping'], verbose=False)]
        )
        
        preds = model.predict(val_data[features])
        lgb_preds.append(preds)
        
        model.save_model(f"lgb_q{int(quantile*100)}_fold{fold}.txt")
    
    lgb_preds = np.column_stack(lgb_preds)
    lll = calculate_metrics(val_data['FVC'].values, lgb_preds, val_data['Base_FVC_Raw'].values)['lll']
    
    return lgb_preds, lll

# ==========================================
# ENSEMBLE TRAINING
# ==========================================
def train_ensemble():
    df, feature_cols, scale_cols = preprocess_data_no_leakage(CONFIG)
    patients = df['Patient'].unique()
    kf = KFold(n_splits=CONFIG['n_folds'], shuffle=True, random_state=CONFIG['seed'])
    
    oof_nn_preds = []
    oof_lgb_preds = []
    oof_trues = []
    oof_baselines = []
    
    print(f"\n{'='*80}")
    print(f"🚀 PROFESSIONAL ENSEMBLE: NEURAL NET + LIGHTGBM")
    print(f"{'='*80}\n")
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(patients)):
        print(f"{'='*80}")
        print(f"FOLD {fold+1}/5")
        print(f"{'='*80}")
        
        train_p, val_p = patients[train_idx], patients[val_idx]
        train_data = df[df['Patient'].isin(train_p)].copy()
        val_data = df[df['Patient'].isin(val_p)].copy()
        
        print(f"\n🧠 Training Neural Network...")
        nn_preds, nn_lll = train_nn_fold(train_data, val_data, feature_cols, scale_cols, fold+1)
        print(f"   NN  - LLL: {nn_lll:.4f}")
        
        print(f"\n🌳 Training LightGBM...")
        lgb_preds, lgb_lll = train_lgb_fold(train_data, val_data, feature_cols, fold+1)
        print(f"   LGB - LLL: {lgb_lll:.4f}")
        
        # Optimize ensemble weights for this fold
        best_lll = -float('inf')
        best_weight = 0.5
        
        for w_nn in np.arange(0.3, 0.8, 0.05):
            w_lgb = 1 - w_nn
            ensemble_preds = w_nn * nn_preds + w_lgb * lgb_preds
            lll = calculate_metrics(val_data['FVC'].values, ensemble_preds, val_data['Base_FVC_Raw'].values)['lll']
            
            if lll > best_lll:
                best_lll = lll
                best_weight = w_nn
        
        final_preds = best_weight * nn_preds + (1 - best_weight) * lgb_preds
        metrics = calculate_metrics(val_data['FVC'].values, final_preds, val_data['Base_FVC_Raw'].values)
        
        print(f"\n🔀 Ensemble (NN:{best_weight:.2f}, LGB:{1-best_weight:.2f})")
        print(f"   LLL:  {metrics['lll']:.4f}  {'✅' if metrics['lll'] > -6.64 else '❌'}")
        print(f"   R²:   {metrics['r2']:.4f}")
        print(f"   RMSE: {metrics['rmse']:.2f} mL")
        print(f"   MAE:  {metrics['mae']:.2f} mL\n")
        
        oof_nn_preds.append(nn_preds)
        oof_lgb_preds.append(lgb_preds)
        oof_trues.extend(val_data['FVC'].values)
        oof_baselines.extend(val_data['Base_FVC_Raw'].values)
    
    # Final ensemble optimization
    oof_nn_preds = np.vstack(oof_nn_preds)
    oof_lgb_preds = np.vstack(oof_lgb_preds)
    oof_trues = np.array(oof_trues)
    oof_baselines = np.array(oof_baselines)
    
    print(f"{'='*80}")
    print(f"🎯 FINAL ENSEMBLE OPTIMIZATION")
    print(f"{'='*80}\n")
    
    best_final_lll = -float('inf')
    best_final_weight = 0.5
    
    for w_nn in np.arange(0.3, 0.8, 0.05):
        w_lgb = 1 - w_nn
        ensemble_preds = w_nn * oof_nn_preds + w_lgb * oof_lgb_preds
        metrics = calculate_metrics(oof_trues, ensemble_preds, oof_baselines)
        
        print(f"NN:{w_nn:.2f} LGB:{w_lgb:.2f} → LLL: {metrics['lll']:.4f}, RMSE: {metrics['rmse']:.2f}")
        
        if metrics['lll'] > best_final_lll:
            best_final_lll = metrics['lll']
            best_final_weight = w_nn
    
    final_ensemble_preds = best_final_weight * oof_nn_preds + (1 - best_final_weight) * oof_lgb_preds
    final_metrics = calculate_metrics(oof_trues, final_ensemble_preds, oof_baselines)
    
    print(f"\n{'='*80}")
    print(f"🏆 FINAL ENSEMBLE RESULTS")
    print(f"{'='*80}")
    print(f"Optimal Weights: NN={best_final_weight:.2f}, LGB={1-best_final_weight:.2f}\n")
    print(f"LLL:  {final_metrics['lll']:.4f}   {'✅' if final_metrics['lll'] > -6.64 else '❌'} (Target > -6.64)")
    print(f"R²:   {final_metrics['r2']:.4f}   {'✅' if final_metrics['r2'] > 0.88 else '❌'} (Target > 0.88)")
    print(f"RMSE: {final_metrics['rmse']:.2f} mL")
    print(f"MAE:  {final_metrics['mae']:.2f} mL")
    print(f"MSE:  {final_metrics['mse']:.2f}")
    
    print(f"\n{'='*80}")
    print(f"💾 MODELS SAVED")
    print(f"{'='*80}")
    print(f"Neural Networks: nn_fold1.pth ... nn_fold5.pth")
    print(f"LightGBM: lgb_q20_fold1.txt ... lgb_q80_fold5.txt")
    print(f"{'='*80}\n")
    
    return final_metrics

if __name__ == "__main__":
    print("="*80)
    print("🛡️ LEAK-FREE PROFESSIONAL ENSEMBLE")
    print("="*80)
    print("✅ Neural Network: Deep MLP with quantile regression")
    print("✅ LightGBM: Gradient boosting for non-linear patterns")
    print("✅ Dynamic weight optimization per fold")
    print("✅ Zero data leakage guarantee")
    print("="*80 + "\n")
    
    results = train_ensemble()
    
    print("\n" + "="*80)
    print("🎯 TARGET STATUS")
    print("="*80)
    
    if results['rmse'] < 170 and results['lll'] > -6.64:
        print("🎉 SUCCESS! Both targets met!")
    elif results['rmse'] < 180:
        print("⚠️ Very close! Consider multi-seed ensemble.")
    else:
        print("📊 Honest baseline established. Further improvements needed.")
    
    print("="*80)

🛡️ LEAK-FREE PROFESSIONAL ENSEMBLE
✅ Neural Network: Deep MLP with quantile regression
✅ LightGBM: Gradient boosting for non-linear patterns
✅ Dynamic weight optimization per fold
✅ Zero data leakage guarantee

✅ Preprocessing: 1549 samples, 24 features

🚀 PROFESSIONAL ENSEMBLE: NEURAL NET + LIGHTGBM

FOLD 1/5

🧠 Training Neural Network...
   NN  - LLL: -6.4964

🌳 Training LightGBM...
   LGB - LLL: -6.4175

🔀 Ensemble (NN:0.30, LGB:0.70)
   LLL:  -6.4121  ✅
   R²:   0.9388
   RMSE: 194.48 mL
   MAE:  127.17 mL

FOLD 2/5

🧠 Training Neural Network...
   NN  - LLL: -6.7391

🌳 Training LightGBM...
   LGB - LLL: -6.6054

🔀 Ensemble (NN:0.30, LGB:0.70)
   LLL:  -6.6098  ✅
   R²:   0.9309
   RMSE: 234.28 mL
   MAE:  155.60 mL

FOLD 3/5

🧠 Training Neural Network...
   NN  - LLL: -6.7747

🌳 Training LightGBM...
   LGB - LLL: -6.5784

🔀 Ensemble (NN:0.30, LGB:0.70)
   LLL:  -6.6083  ✅
   R²:   0.9121
   RMSE: 222.96 mL
   MAE:  151.09 mL

FOLD 4/5

🧠 Training Neural Network...
   NN  - LLL: -6